# Crawling Website PTA Trunojoyo

## Install Library BeautifulSoup

In [1]:
!pip install beautifulsoup4

## Import Library

In [2]:
from bs4 import BeautifulSoup
import requests
import csv

## Create Crawling Function

In [3]:
def get_data(url):
  data_page = []
  response = requests.get(url)
  if response.status_code == 200:
    page = BeautifulSoup(response.content)
    container_content = page.find('li', {"data-id" : "id-1"})
    header_page = container_content.find('div', {"style" : "float:left; width:540px;"})

    #  Mengambil judul TA
    get_title = header_page.find('a', class_ = 'title').text.strip().split('\r\n')
    title = " ".join(get_title)
    data_page.append(title)

    # Mengambil Penulis dan juga Dospem
    creator = header_page.find_all('span')
    for i in creator:
      splitting_text = i.text.strip().split(':')
      if splitting_text[0].strip() == 'Penulis':
        data_page.append(splitting_text[1].strip())
      elif splitting_text[0].strip() == 'Dosen Pembimbing I':
        data_page.append(splitting_text[1].strip())
      else:
        data_page.append(splitting_text[1].strip())

    # Mengambil Abstrak
    get_abstrak = container_content.find('p', {"align" : "justify"}).text.strip().split('\r\n')
    abstrak = " ".join(get_abstrak)
    data_page.append(abstrak)

  return data_page


def crawling_pta(url):
  hasil_crawling = []
  response = requests.get(url)
  if response.status_code == 200:
    page = BeautifulSoup(response.content)
    container_content = page.find('ul', class_= 'items')
    list_content = container_content.find_all('li')

    for content in list_content:
      view_button = content.find('a', class_ = 'gray button').get('href')
      hasil_crawling.append(get_data(view_button))

    next_page = page.find_all('a', class_='pag_button')
    for i in next_page:
      if i.text.strip() == '>':
        return (hasil_crawling, i.get('href'))

def main(url, num_page):
  content, next_page = crawling_pta(url)

  for i in range(num_page-1):
    more_content, next_page = crawling_pta(next_page)
    content += more_content

  with open('crawling_pta.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Judul', 'Penulis', 'Dosen Pembimbing I', 'Dosen Pembimbing II', 'Abstrak'])
    writer.writerows(content)

  print(f'Data Berhasil Di Simpan Kedalam File crawling_pta.csv Dengan Jumlah : {len(content)} Data')

main('https://pta.trunojoyo.ac.id/c_search/byprod/10', 4)

Data Berhasil Di Simpan Kedalam File crawling_pta.csv Dengan Jumlah : 20 Data
